In [1]:
import os
import pandas as pd
from langchain_ollama import ChatOllama
from langchain.agents import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
os.environ["OPENAI_API_KEY"] = ""

os.environ["GOOGLE_API_KEY"] = ""

In [ ]:
df = pd.read_csv("finans.csv")

idx = 0

chunk = df.iloc[idx:idx+1]
chunk

PROMPTS

In [4]:
agent_prefix = """
You are an expert data analyst. I will provide you with one row of data from a dataframe, containing several metrics. For each metric in the row, perform a thorough analysis and provide the following details for each:

1. **Metric Name**: The name of the metric.
2. **Trend**: Describe the overall trend of the metric (e.g., increasing, decreasing, stable, fluctuating).
3. **Change Rate**: Provide the rate of change over time for the metric (e.g., percentage increase or decrease, or "N/A" if not applicable).
4. **Current Status**: Summarize the current state of the metric (e.g., high, low, average).
5. **Anomalies**: Indicate whether there are any anomalies in the metric and provide a description if anomalies are detected.
6. **Summary**: Summarize the key findings for the metric in a brief statement.

Ensure that each metric is analyzed thoroughly, and respond with raw, unfiltered output for all metrics in this format:

- Metric Name: [Name]
- Trend: [Trend]
- Change Rate: [Rate]
- Current Status: [Status]
- Anomalies: [True/False, Description if True]
- Summary: [Summary]

"""

In [5]:
output_format_prompt_string = """
Reorganize the given text according to the output format {format_instructions}.

The given text is:
{response}
"""


output_prompt = PromptTemplate(
    input_variables=["format_instructions", "response"],
    template=output_format_prompt_string
)

In [6]:
class MetricAnalysis(BaseModel):
    metric_name: str = Field(description="The name of the metric being analyzed. Example: 'Revenue', 'User Growth Rate', 'Profit Margin'")
    trend: str = Field(description="The overall trend of the metric. Example: 'Increasing', 'Decreasing', 'Stable', 'Fluctuating'")
    change_rate: str = Field(description="The rate of change over time for the metric. Example: '10% increase', '5% decrease', 'N/A'")
    current_status: str = Field(description="The current status of the metric. Example: 'High', 'Low', 'Average'")
    anomalies: bool = Field(description="Whether any anomalies are detected in the metric. Example: True, False")
    summary: str = Field(description="A brief summary of the key findings for the metric.")

parser = PydanticOutputParser(pydantic_object=MetricAnalysis)

In [7]:
query = """Analyze the following metrics:"""

In [8]:
llm_openai = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm_gemini = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
llm_gemma = ChatOllama(model="gemma2:9b", temperature=0)

In [ ]:
# OPENAI + GEMMA

agent = create_pandas_dataframe_agent(
    llm_gemma,
    chunk,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True,
    max_iterations=3,
    prefix=agent_prefix,
)

try:
    first_response = agent.invoke(query)["output"]
except Exception as e:
    first_response = str(e) 

In [ ]:
first_response

In [ ]:

chain = output_prompt | llm_gemma | parser 

second_response = chain.invoke({"format_instructions": parser.get_format_instructions(), "response": first_response})

print(second_response)

In [ ]:
#GEMINI + GEMMA

agent = create_pandas_dataframe_agent(
    llm_gemini,
    chunk,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True,
    max_iterations=3,
    prefix=agent_prefix,
)

try:
    first_response = agent.invoke(query)["output"]
except Exception as e:
    first_response = str(e) 

chain = output_prompt | llm_gemma | parser 

second_response = chain.invoke({"format_instructions": parser.get_format_instructions(), "response": first_response})

print(second_response)

In [ ]:
#GEMMA + GEMMA

agent = create_pandas_dataframe_agent(
    llm_gemma,
    chunk,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True,
    max_iterations=3,
    prefix=agent_prefix,
)

try:
    first_response = agent.invoke(query)["output"]
except Exception as e:
    first_response = str(e) 

chain = output_prompt | llm_gemma | parser 

second_response = chain.invoke({"format_instructions": parser.get_format_instructions(), "response": first_response})

print(second_response)